In [20]:
# 기본세팅
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
import random

# 크롤링용
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 컬럼, 로우 조회 최대수 지정 
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

# 시각화할때 warning 무시
import warnings
warnings.filterwarnings(action='ignore')

# 정보
- 일단 탑10밖에 안된다.
- list 만들어서 그냥 탑텐정도 뽑아놓고 셀레니움을 해보자

In [21]:
# 지역 및 가게별 검색어 리스트를 따로 만들어보자
sgg = pd.read_csv('/Users/ppangppang/Desktop/ssac/data_crwaling/restaurant_test/SGG.csv')
sgg['county'] = sgg['county'].apply(lambda x:'부산' if (x == "부산광역시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'서울' if (x == "서울특별시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'경북' if (x == "경상북도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'경남' if (x == "경상남도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'전북' if (x == "전라북도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'전남' if (x == "전라남도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'전북' if (x == "전라북도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'충북' if (x == "충청북도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'충남' if (x == "충청북도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'대구' if (x == "대구광역시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'울산' if (x == "울산광역시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'제주' if (x == "제주특별자치도") else x)
sgg['county'] = sgg['county'].apply(lambda x:'세종' if (x == "세종특별자치시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'인천' if (x == "인천광역시") else x)
sgg['county'] = sgg['county'].apply(lambda x:'광주' if (x == "광주광역시") else x)

In [22]:
list_df=sgg[(sgg['county']=='제주')]
# list_df.drop(['si_gun'],axis=1,inplace=True)
# list_df=list_df.dropna()

list_gu = list(list_df['gu'])

# list_gu1 = []
# for i in list_gu:
#     if i != '부산진구':
#         list_gu1.append('부산'+i)
#     else:
#         list_gu1.append(i)
        
# list_gu2=[]
# for i in list_gu1:
#     if i not in list_gu2:
#         list_gu2.append(i)

In [23]:
list_df['dong'].unique()

array(['일도일동', '일도이동', '이도일동', '이도이동', '삼도일동', '삼도이동', '건입동', '용담일동',
       '용담이동', '용담삼동', '화북일동', '화북이동', '삼양일동', '삼양이동', '삼양삼동', '봉개동',
       '아라일동', '아라이동', '오라일동', '오라이동', '오라삼동', '노형동', '외도일동', '외도이동',
       '이호일동', '이호이동', '도두일동', '도두이동', '도남동', '도련일동', '도련이동', '용강동',
       '회천동', '오등동', '월평동', '영평동', '연동', '도평동', '해안동', '내도동', '한림읍',
       '귀덕리', '수원리', '대림리', '한림리', '금악리', '동명리', '명월리', '옹포리', '상대리',
       '상명리', '협재리', '월령리', '금능리', '월림리', '한수리', '애월읍', '용흥리', '중엄리',
       '광령리', '상귀리', '구엄리', '수산리', '신엄리', '장전리', '소길리', '고성리', '하가리',
       '상가리', '고내리', '애월리', '납읍리', '어음리', '곽지리', '금성리', '봉성리', '하귀리',
       '유수암리', '구좌읍', '동복리', '덕천리', '송당리', '월정리', '행원리', '세화리', '상도리',
       '한동리', '하도리', '종달리', '평대리', '김녕리', '조천읍', '신촌리', '와흘리', '조천리',
       '함덕리', '신흥리', '대흘리', '북촌리', '와산리', '교래리', '선흘리', '한경면', '청수리',
       '용수리', '고산리', '판포리', '금등리', '조수리', '두모리', '신창리', '낙천리', '저지리',
       '산양리', '한원리', '용당리', '추자면', '대서리', '영흥리', '묵리', '예초리', '신양리',
       '우도면', '연평리', '

In [24]:
list_gu2

NameError: name 'list_gu2' is not defined

In [ ]:
# 음식 리스트 가져오기
food_cat = pd.read_excel('/Users/ppangppang/Desktop/ssac/data_crwaling/restaurant_test/food_cat.xlsx')
# food_cat.drop(columns='Unnamed: 0',axis=1, inplace=True)
food = (list(food_cat['list']))

In [25]:
# test
food1=food
target=[]
for loca in list_gu2[11:12]:
    for foody in food1:
        target.append(loca + foody)

NameError: name 'food' is not defined

In [ ]:
target

In [26]:
# 음식이름으로 검색
search_list = target

restaurant_df = pd.DataFrame()
date=str(datetime.date.today()).replace('-','')[2:]

for f_name in search_list:
    url = f'https://www.diningcode.com/list.php?query={f_name}'
    
    seed = np.random.randint(100)
    np.random.seed(seed)    
    a = np.random.randint(5)
    time.sleep(a)
    try:
        response = requests.get(url) 
    except:
        print('page error')
        break
        
    data = BeautifulSoup(response.text, 'html.parser')

    # 순위제거(상황봐서 순위 넣으려면 반복문만 주석처리)
    store = [name.text for name in data.select('span.btxt')]
    for i in range(len(store)):
        store[i] = store[i][3:]

    # 주소정보
    summary = [s_mary.text for s_mary in data.select('span.stxt')]
    sub_loca = [sub.text for sub in data.select('i.loca')]
    full_loca = [full_l.text for full_l in data.select('span.ctxt')][1::2]

    # full_loca 중복처리
    try:
        for i in range(len(full_loca)):
            if full_loca[i][0:len(sub_loca[i])] == sub_loca[i]:
                full_loca[i] = full_loca[i][len(sub_loca[i]):]
            else:
                pass
    except: pass
    
    # 카테고리 지정
    category = [f_name] * len(full_loca)
    food_cat = [f_name[3:]] * len(full_loca)
    
    # 점수
    score = [point.text for point in data.select('span.point')]

    cat_df = pd.DataFrame(columns=['category','food_cat','store','score','summary','sub_loca','full_loca'], data=list(zip(category,food_cat,store,score,summary,sub_loca,full_loca)))
    temp = cat_df
    restaurant_df = pd.concat([restaurant_df,temp])
    restaurant_df.index=range(1,len(restaurant_df)+1)
    print(f'{f_name} 크롤링 완료')

print('\n *****크롤링완료*****')

# test
restaurant_df.to_csv(f'/Users/ppangppang/Desktop/맛집리스트확인/{date}_restaurant_list3.csv',index=False)

# 데이터저장
restaurant_df.to_csv(f'/Users/ppangppang/Documents/dev/TIL/test_restaurant_data/{date}_restaurant_list3.csv',index=False)
# 백업
restaurant_df.to_csv(f'/Users/ppangppang/Desktop/ssac/data_crwaling/restaurant_test/databackup/{date}_restaurant_list3.csv',index=False)

NameError: name 'target' is not defined

In [25]:
restaurant_df

,category,food_cat,store,score,summary,sub_loca,full_loca
1,부산강서구가라아게,서구가라아게,키친어썸,,"사시미, 술집, 치킨가라아게",명지,부산광역시 강서구 명지동 3357-2
2,부산강서구가리비,서구가리비,수성수산,33점,"조개구이, 조개찜, 가리비",부산,부산광역시 강서구 신호동 321-1
3,부산강서구가리비,서구가리비,불타는조개구이,32점,"조개구이, 갈미조개, 가리비",명지,부산광역시 강서구 명지동 1526-7
4,부산강서구가리비,서구가리비,시소,,"해산물, 조개구이, 가리비",강서구,부산광역시 강서구 신호동 220-7
5,부산강서구가리비,서구가리비,수연횟집,,"생선회, 가리비",부산강서구,부산광역시 강서구 녹산동 60-4
...,...,...,...,...,...,...,...
3607,부산강서구히쯔마부시,서구히쯔마부시,심해,33점,"곰장어, 장어, 장어덮밥",명지,부산광역시 강서구 명지동 3239
3608,부산강서구히쯔마부시,서구히쯔마부시,뚜레박 장어구이,32점,"장어구이, 장어, 장어덮밥",김해,경상남도 김해시 불암동 229-21
3609,부산강서구히쯔마부시,서구히쯔마부시,튜나테이블,,"참치전문, 생참치, 장어덮밥",명지,부산광역시 강서구 명지동 3237-3
3610,부산강서구히쯔마부시,서구히쯔마부시,백년장어,,"장어, 장어덮밥",김해,경상남도 김해시 불암동 229-16


In [26]:
df = pd.read_csv(f'/Users/ppangppang/Desktop/맛집리스트확인/{date}_restaurant_list3.csv')
df.drop_duplicates(['store'],inplace=True)
df.index = range(len(df))
df

,category,food_cat,store,score,summary,sub_loca,full_loca
0,부산강서구가라아게,서구가라아게,키친어썸,NaN,"사시미, 술집, 치킨가라아게",명지,부산광역시 강서구 명지동 3357-2
1,부산강서구가리비,서구가리비,수성수산,33점,"조개구이, 조개찜, 가리비",부산,부산광역시 강서구 신호동 321-1
2,부산강서구가리비,서구가리비,불타는조개구이,32점,"조개구이, 갈미조개, 가리비",명지,부산광역시 강서구 명지동 1526-7
3,부산강서구가리비,서구가리비,시소,NaN,"해산물, 조개구이, 가리비",강서구,부산광역시 강서구 신호동 220-7
4,부산강서구가리비,서구가리비,수연횟집,NaN,"생선회, 가리비",부산강서구,부산광역시 강서구 녹산동 60-4
...,...,...,...,...,...,...,...
891,부산강서구활어,서구활어,동해초장,31점,"대게, 킹크랩, 회센터",신호동,부산광역시 강서구 신호동 317-18
892,부산강서구회,서구회,동해대게,31점,"대게, 킹크랩, 활어회",부산대,부산광역시 강서구 신호동 318-2
893,부산강서구회전초밥,서구회전초밥,갓덴스시 스타필드시티명지점,NaN,"회전초밥, 스시",스타필드시티명지,부산광역시 강서구 명지동 3438 스타필드시티명지점
894,부산강서구후라이드치킨,서구후라이드치킨,맥시칸,NaN,"치킨, 닭강정, 후라이드치킨",범방동,부산광역시 강서구 범방동 1867-7


In [36]:
# 상위점수 구분(30점 이상만)
# a = list(df['score'])
# for i in range(len(a)):
#     a[i] = int(a[i][:-1])
# df['re_score'] = a
# df = df[df['re_score']>=30]
# df.index=range(len(df))
# df.drop(['re_score'],axis=1,inplace=True)
# df=df.sort_values('score',ascending=False)

In [27]:
df.to_excel('/Users/ppangppang/Desktop/맛집리스트확인/select/부산강서구.xlsx',index=False)

In [13]:
# df.to_csv('/Users/ppangppang/Documents/dev/TIL/test_restaurant_data/211010_restaurant_list3.csv',index=False)

## 현재까지 작업한 파일 concat

In [22]:
import glob
import pandas as pd

# get data file names
filenames = glob.glob('/Users/ppangppang/Desktop/ssac/data_crwaling/restaurant_test/databackup/*.csv')

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)
df.drop_duplicates(['store'],inplace=True)
df.index = range(len(df))

In [23]:
df

,category,food_cat,store,score,summary,sub_loca,full_loca
0,부평냉면,면,해민면옥,61점,"평양냉면, 녹두전, 냉면",부평,인천광역시 부평구 부평동 212-24
1,부평냉면,면,부평막국수,56점,"수육, 물막국수, 냉면",부평시장역,인천광역시 부평구 부평1동 529-96
2,부평냉면,면,별미칡냉면,54점,"냉면, 물냉면",부평,인천시 부평구 갈산동 377-8
3,부평냉면,면,화미소금구이 부평점,41점,"삼겹살, 목살, 냉면",부평,인천광역시 부평구 부평동 70-83
4,부평냉면,면,디델리,35점,"떡볶이, 라볶이, 물냉면",부평,인천광역시 부평구 부평동 211-75
...,...,...,...,...,...,...,...
48631,계양찹쌀꽈배기,쌀꽈배기,노랑고래 효성점,NaN,"도너츠, 꽈배기, 찹쌀꽈배기",효성동,인천광역시 계양구 효성동 242-7
48632,계양치즈핫도그,즈핫도그,소문난 마늘숙성 수제돈가스,NaN,"돈까스, 수제돈까스, 치즈핫도그",효성동,인천광역시 계양구 병방동 373 28호
48633,계양닭다리,다리,BHC치킨 임학점,NaN,"치킨, 치즈볼, 닭다리",임학역,인천광역시 계양구 임학동 65-11
48634,계양커피콩빵,피콩빵,투씨커피,NaN,"카페, 커피, 커피콩빵",계산동,인천광역시 계양구 작전동 905-1


In [24]:
df.isna().sum()

category         0
food_cat      1534
store            0
score        12268
summary          2
sub_loca         0
full_loca        0
dtype: int64

In [25]:
# 어차피 0점은 없음. 점수가 있거나 없거나임
df['score']=df['score'].fillna('0점')
df

,category,food_cat,store,score,summary,sub_loca,full_loca
0,부평냉면,면,해민면옥,61점,"평양냉면, 녹두전, 냉면",부평,인천광역시 부평구 부평동 212-24
1,부평냉면,면,부평막국수,56점,"수육, 물막국수, 냉면",부평시장역,인천광역시 부평구 부평1동 529-96
2,부평냉면,면,별미칡냉면,54점,"냉면, 물냉면",부평,인천시 부평구 갈산동 377-8
3,부평냉면,면,화미소금구이 부평점,41점,"삼겹살, 목살, 냉면",부평,인천광역시 부평구 부평동 70-83
4,부평냉면,면,디델리,35점,"떡볶이, 라볶이, 물냉면",부평,인천광역시 부평구 부평동 211-75
...,...,...,...,...,...,...,...
48631,계양찹쌀꽈배기,쌀꽈배기,노랑고래 효성점,0점,"도너츠, 꽈배기, 찹쌀꽈배기",효성동,인천광역시 계양구 효성동 242-7
48632,계양치즈핫도그,즈핫도그,소문난 마늘숙성 수제돈가스,0점,"돈까스, 수제돈까스, 치즈핫도그",효성동,인천광역시 계양구 병방동 373 28호
48633,계양닭다리,다리,BHC치킨 임학점,0점,"치킨, 치즈볼, 닭다리",임학역,인천광역시 계양구 임학동 65-11
48634,계양커피콩빵,피콩빵,투씨커피,0점,"카페, 커피, 커피콩빵",계산동,인천광역시 계양구 작전동 905-1


In [26]:
df[df['summary'].isna()]

,category,food_cat,store,score,summary,sub_loca,full_loca
39192,종로구운니동,NaN,COFFEE la,21점,NaN,안국역,서울시 종로구 운니동 15 노블호텔1층
39836,강서구공항동,NaN,호호카츠 김포공항점,26점,NaN,김포공항,서울특별시 강서구 공항동 1373 국내선청사 4층


In [39]:
# df['summary'][39836:39837] = '돈까스'
# df['summary'][39192:39293] = '커피'

In [40]:
df.isna().sum()

category        0
food_cat     1534
store           0
score           0
summary         0
sub_loca        0
full_loca       0
dtype: int64

In [47]:
# food_cat은 부가정보로 남겨둔거니까 패스
df = df[['category','store','summary','sub_loca','full_loca']]
df.index = range(len(df))
df

,category,store,summary,sub_loca,full_loca
0,부평냉면,해민면옥,"평양냉면, 녹두전, 냉면",부평,인천광역시 부평구 부평동 212-24
1,부평냉면,부평막국수,"수육, 물막국수, 냉면",부평시장역,인천광역시 부평구 부평1동 529-96
2,부평냉면,별미칡냉면,"냉면, 물냉면",부평,인천시 부평구 갈산동 377-8
3,부평냉면,화미소금구이 부평점,"삼겹살, 목살, 냉면",부평,인천광역시 부평구 부평동 70-83
4,부평냉면,디델리,"떡볶이, 라볶이, 물냉면",부평,인천광역시 부평구 부평동 211-75
...,...,...,...,...,...
48631,계양찹쌀꽈배기,노랑고래 효성점,"도너츠, 꽈배기, 찹쌀꽈배기",효성동,인천광역시 계양구 효성동 242-7
48632,계양치즈핫도그,소문난 마늘숙성 수제돈가스,"돈까스, 수제돈까스, 치즈핫도그",효성동,인천광역시 계양구 병방동 373 28호
48633,계양닭다리,BHC치킨 임학점,"치킨, 치즈볼, 닭다리",임학역,인천광역시 계양구 임학동 65-11
48634,계양커피콩빵,투씨커피,"카페, 커피, 커피콩빵",계산동,인천광역시 계양구 작전동 905-1


In [79]:
# 일단 서울, 인천만보자
location = pd.read_csv('/Users/ppangppang/Desktop/ssac/data_crwaling/restaurant_test/adm_code.csv')
location = location[(location['시도코드'] == 23) | (location['시도코드'] == 11)]
location.index = range(len(location))
location

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭,전체코드
0,11,서울특별시,11010,종로구,1101053,사직동,11110101101053
1,11,서울특별시,11010,종로구,1101054,삼청동,11110101101054
2,11,서울특별시,11010,종로구,1101055,부암동,11110101101055
3,11,서울특별시,11010,종로구,1101056,평창동,11110101101056
4,11,서울특별시,11010,종로구,1101057,무악동,11110101101057
...,...,...,...,...,...,...,...
574,23,인천광역시,23320,옹진군,2332033,백령면,23233202332033
575,23,인천광역시,23320,옹진군,2332034,대청면,23233202332034
576,23,인천광역시,23320,옹진군,2332035,덕적면,23233202332035
577,23,인천광역시,23320,옹진군,2332036,자월면,23233202332036


In [77]:
df2 = df[df['full_loca'].str.contains('임학동')]

In [83]:
# df['si'] = np.where(df['full_loca'].str.contains('서울'), 11, 23)

In [84]:
df['si'] = np.where(df['full_loca'].str.contains('서울'), '서울', '인천')

,category,store,summary,sub_loca,full_loca,si
0,부평냉면,해민면옥,"평양냉면, 녹두전, 냉면",부평,인천광역시 부평구 부평동 212-24,23
1,부평냉면,부평막국수,"수육, 물막국수, 냉면",부평시장역,인천광역시 부평구 부평1동 529-96,23
2,부평냉면,별미칡냉면,"냉면, 물냉면",부평,인천시 부평구 갈산동 377-8,23
3,부평냉면,화미소금구이 부평점,"삼겹살, 목살, 냉면",부평,인천광역시 부평구 부평동 70-83,23
4,부평냉면,디델리,"떡볶이, 라볶이, 물냉면",부평,인천광역시 부평구 부평동 211-75,23
...,...,...,...,...,...,...
48631,계양찹쌀꽈배기,노랑고래 효성점,"도너츠, 꽈배기, 찹쌀꽈배기",효성동,인천광역시 계양구 효성동 242-7,23
48632,계양치즈핫도그,소문난 마늘숙성 수제돈가스,"돈까스, 수제돈까스, 치즈핫도그",효성동,인천광역시 계양구 병방동 373 28호,23
48633,계양닭다리,BHC치킨 임학점,"치킨, 치즈볼, 닭다리",임학역,인천광역시 계양구 임학동 65-11,23
48634,계양커피콩빵,투씨커피,"카페, 커피, 커피콩빵",계산동,인천광역시 계양구 작전동 905-1,23


In [71]:
location[location['읍면동명칭'] == '연희동']

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭,전체코드
205,11,서울특별시,11130,서대문구,1113076,연희동,11111301113076
517,23,인천광역시,23080,서구,2308053,연희동,23230802308053


In [304]:
#예제 데이터 만들기
ex = {'경도' : [127.061026,127.047883,127.899220,128.980455,127.104071,127.102490,127.088387,126.809957,127.010861,126.836078
                ,127.014217,126.886859,127.031702,126.880898,127.028726,126.897710,126.910288,127.043189,127.071184,127.076812
                ,127.045022,126.982419,126.840285,127.115873,126.885320,127.078464,127.057100,127.020945,129.068324,129.059574
                ,126.927655,127.034302,129.106330,126.980242,126.945099,129.034599,127.054649,127.019556,127.053198,127.031005
                ,127.058560,127.078519,127.056141,129.034605,126.888485,129.070117,127.057746,126.929288,127.054163,129.060972],
     '위도' : [37.493922,37.505675,37.471711,35.159774,37.500249,37.515149,37.549245,37.562013,37.552153,37.538927,37.492388
              ,37.480390,37.588485,37.504067,37.608392,37.503693,37.579029,37.580073,37.552103,37.545461,37.580196,37.562274
              ,37.535419,37.527477,37.526139,37.648247,37.512939,37.517574,35.202902,35.144776,37.499229,35.150069,35.141176
              ,37.479403,37.512569,35.123196,37.546718,37.553668,37.488742,37.493653,37.498462,37.556602,37.544180,35.111532
              ,37.508058,35.085777,37.546103,37.483899,37.489299,35.143421],
     '구분' : ['음식','음식','음식','음식','생활서비스','음식','음식','음식','음식','음식','음식','음식','음식','음식','음식'
             ,'음식','음식','소매','음식','음식','음식','음식','소매','음식','소매','음식','음식','음식','음식','음식','음식'
             ,'음식','음식','음식','음식','소매','음식','음식','의료','음식','음식','음식','소매','음식','음식','음식','음식'
             ,'음식','음식','음식']}

ex=pd.DataFrame(ex)

In [324]:
lat = ex.loc[0,'위도']
long = ex.loc[0,'경도']

In [ ]:
#지도 새로 띄우기
m = folium.Map([lat,long],zoom_start=12)

for i in ex.index:
    sub_lat =  ex.loc[i,'위도']
    sub_long = ex.loc[i,'경도']
    
    title = ex.loc[i,'구분']
  
    #구분이 소매면 빨간색으로 표시, default는 녹색
    color = 'green'
    if ex.loc[i,'구분'] == '소매':
        color = "red"
        
    #지도에 동그라미로 데이터 찍기    
    folium.CircleMarker([sub_lat,sub_long],color=color,radius = 5, tooltip=title).add_to(m)

m

In [329]:
driver = webdriver.Chrome('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/chromedriver')

url = 'https://www.google.com/maps/'
driver.get(url)
time.sleep(5)


1. 인기 메뉴 카테고리 설정(프렌차이즈가 많으니, 적어도 50분야 정도 설정)
2. 구단위 뒤도 경도 뽑아보기(좀 어려우면 좀 묶어서)
3. 카테고리별 맛집 평균점수에따라 지도에 표시해보기
여기서 나올만한 인사이트? - 카테고리별 지역별 맛집 분포 / 배달과 관련해서 뭐 엮을만한게 있을까? / 시간되면 생활지수들 질병관리청에서 가져와서 상관관계 확인?

위도경도 링크1 : https://sajuplus.tistory.com/778
위도경도 링크2 : https://kor.timegenie.com/latitude_longitude/country/kr
위도경도 api : https://www.vworld.kr/dev/v4dv_2ddataguide2_s002.do?svcIde=ademd

다른 데이터셋 확인 : kosis , 질병관리청